## Setup

In [7]:
import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']
url_contact = os.environ["URL_CONTACT"]
url_newsletter = os.environ["URL_NEWSLETTER"]

In [8]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain

from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

In [9]:
model_name = "gpt-4"
temperature = 0.7
model = OpenAI(model_name=model_name, temperature=temperature, max_tokens=4097)

/home/chris/anaconda3/envs/ai-dev/lib/python3.11/site-packages/langchain/llms/openai.py:216: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/home/chris/anaconda3/envs/ai-dev/lib/python3.11/site-packages/langchain/llms/openai.py:811: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


## Input

In [10]:
input = "How to play with different color schemes and patterns to give rooms personality."

## Execution

In [11]:
# Define your desired data structure.
class Outline(BaseModel):
    intro: str = Field(description="The introduction to the blog post")
    section1: str = Field(description="The first section of the blog post, not including the introduction")
    section2: str = Field(description="The second section of the blog post, not including the introduction")
    section3: str = Field(description="The third section of the blog post, not including the introduction")
    section4: str = Field(description="The fourth section of the blog post, not including the introduction")
    section5: str = Field(description="The fith section of the blog post, not including the introduction")
    conclusion: str = Field(description="The conclusion of the blog post")
    


In [12]:
# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=Outline)
outline_human_msg = """Please write a blog post with 5 sections in addition to the introduction and conclusion in the first person about "{input}" 

Each section, including the introduction and conclusion, should be at least 3 paragraphs.

{format_instructions}
"""
prompt = PromptTemplate(
    template=outline_human_msg,
    input_variables=["input"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)
print(parser.get_format_instructions())
_input = prompt.format_prompt(input=input)
output = model(_input.to_string())
parsed_output = parser.parse(output)
print(parsed_output)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"intro": {"title": "Intro", "description": "The introduction to the blog post", "type": "string"}, "section1": {"title": "Section1", "description": "The first section of the blog post, not including the introduction", "type": "string"}, "section2": {"title": "Section2", "description": "The second section of the blog post, not including the introduction", "type": "string"}, "section3": {"title": "Section3", "description": "The third section of the blog post, not including the introduction", "type": "string"}, "section4": {"title

In [13]:

system_msg = """You are a world class interior designer that specializes in the Transitional Design style.  
You have a knack for explaining concepts related to interior design in a way that is accessible and engaging.  
You avoid using the passive voice in any sentence. 
Your sentences should be no more than 20 words long."""

def get_section_chain(section_num, model):
    prompt_template = f"""{{system_msg}}
        
            Below is a draft of the short version of a section from a blog post on '{{blog_topic}}'. 
            Please expand the section by writing at least 5 paragraphs on this topic.  
            Ensure that the section has a succint header that summarizes the content of the section with HTML formatting.  
            Please include subheaders within the section where appropriate.
            Section headers should be in the format:
            <h2>Section {section_num}</h2>
            <h3>Subsection</h3>
            <h4>Subsubsection</h4>
            Because this is one section in a larger blog post, do not start any paragraph with "Lastly" or "In Conclusion".  
            This secion merely must transition to the next one.  
            The end of this section is NOT the end of the blog post.
        
        ### SECTION DRAFT  
        {{section_{section_num}}}
        """
    prompt=ChatPromptTemplate.from_template(prompt_template)
        

    return LLMChain(llm=model, prompt=prompt, output_key=f"expanded_section_{section_num}")

def run_section_chain(section_num, section_draft, model=model, system_msg=system_msg, input=input):
    section_chain = get_section_chain(section_num, model)
    return section_chain.run(
            {
            'system_msg': system_msg,
            'blog_topic': input,
            f'section_{section_num}': section_draft,
            }
        )
    



In [14]:
intro_prompt=ChatPromptTemplate.from_template(
    """{system_msg}
    
    Below is a draft of the short version of introduction from a blog post on '{blog_topic}'. 
    Please expand the section by writing at least 3 paragraphs on this topic. 
    Let people know that they can always contact me if they would like advice our have questions by linking to my Contact me page.  Here is the link for that: {url_contact}
    Ensure that the section has a succint header that summarizes the content of the section with HTML formatting.
    The heading for this section should be an <h1> tag.
    Because this is the introduction to the blog post, do not start any paragraph with "Finally", "Lastly" or "In Conclusion".
    This section merely must transition to the next one.  
    The end of this section is NOT the end of the blog post.
   
   
   ### SECTION DRAFT   
   {intro}
    """
)

intro_chain = LLMChain(llm=model, prompt=intro_prompt, output_key="expanded_intro")

expanded_intro = intro_chain.run(
    {
    'system_msg': system_msg,
    'blog_topic': input,
    'intro': parsed_output.intro,
    'url_contact': url_contact,
    }
)
print(expanded_intro)

<h1>Unleashing the Power of Color and Pattern in Your Home</h1>

Colors and patterns can transform your living space in unique ways. Understanding color theory and pattern application informs your design choices. It creates a striking balance between aesthetics and functionality. When you grasp the emotions colors evoke, you can tailor your space to embody your desired atmosphere. For example, blues induce calmness, yellows inspire cheerfulness, and reds ignite passion. By strategically selecting a color scheme, you can create a soothing oasis, a vibrant social space, or a productive work area.

Patterns, like colors, play a crucial role in interior design. They hold the power to add an exciting visual interest, breaking up monotony and tying varying elements together. Patterns can range from bold to subtle, modern to traditional, geometric to floral. The trick lies in finding a harmonious balance that enhances your overall design without overwhelming it.

Remember, it's your space and

In [15]:
expanded_sec1 = run_section_chain(1, parsed_output.section1)
expanded_sec2 = run_section_chain(2, parsed_output.section2)
expanded_sec3 = run_section_chain(3, parsed_output.section3)
expanded_sec4 = run_section_chain(4, parsed_output.section4)
expanded_sec5 = run_section_chain(5, parsed_output.section5)

In [16]:
conclusion_prompt=ChatPromptTemplate.from_template(
    """{system_msg}
     
    Below is a draft of the short version of the conclusion from a blog post on '{blog_topic}'. 
    Please expand the section by writing at least 2 paragraphs on this topic. 
    There should be a <h2> section header indicating that the blog post is ending.
    Also, please include hyperlinks so that readers can contact me for a free consultation or sign up for my newsletter. 
    The link to my newsletter is {url_newsletter}
    The "Contact Me" link should have a URL of {url_contact}
    The format should be HTML.
    
   
   ### SECTION DRAFT   
   {conclusion}
    """
)

conclusion_chain = LLMChain(llm=model, prompt=conclusion_prompt, output_key="expanded_conclusion")

expanded_conclusion = conclusion_chain.run(
    {
    'system_msg': system_msg,
    'blog_topic': input,
    'conclusion': parsed_output.conclusion,
    'url_contact': url_contact,
    'url_newsletter': url_newsletter
    }
)
print(expanded_conclusion)

<h2>Conclusion: Unleashing Your Inner Designer</h2>

Exploring varied color schemes and patterns in your rooms infuses them with life and personality. <a href="https://deborahrucci.decoratingden.com/contact/">Get in touch</a> if you need expert guidance on where to begin. Your understanding of basic color theory and pattern use will help you craft visually captivating, harmonious spaces that are true to your style.

There is no definitive blueprint for design. The most enchanting spaces are the ones that bring you comfort and joy. Therefore, feel free to experiment, bend the rules, and design a home that is distinctly yours. 

May these guidelines inspire you to delve into the world of color and pattern in your home. Enjoy the process of decorating! Remember, if you need further inspiration or tips, you can <a href="https://deborahrucci.decoratingden.com/newsletter/">sign up for my newsletter</a>. Happy designing!


In [17]:
full_post = f"""{expanded_intro}

{expanded_sec1}

{expanded_sec2}

{expanded_sec3}

{expanded_sec4}

{expanded_sec5}

{expanded_conclusion}

"""




In [18]:
seo_prompt = ChatPromptTemplate.from_template("""
Examine this blog post and make sure that it is SEO friendly. 
Please list a good seo keyphrase and a meta description.

BLOG POST:
{full_post}
""")

seo_chain = LLMChain(llm=model, prompt=seo_prompt)

seo_vars = seo_chain.run(
    {
    'full_post': full_post
    }
)


## Output

In [19]:
print(full_post)

<h1>Unleashing the Power of Color and Pattern in Your Home</h1>

Colors and patterns can transform your living space in unique ways. Understanding color theory and pattern application informs your design choices. It creates a striking balance between aesthetics and functionality. When you grasp the emotions colors evoke, you can tailor your space to embody your desired atmosphere. For example, blues induce calmness, yellows inspire cheerfulness, and reds ignite passion. By strategically selecting a color scheme, you can create a soothing oasis, a vibrant social space, or a productive work area.

Patterns, like colors, play a crucial role in interior design. They hold the power to add an exciting visual interest, breaking up monotony and tying varying elements together. Patterns can range from bold to subtle, modern to traditional, geometric to floral. The trick lies in finding a harmonious balance that enhances your overall design without overwhelming it.

Remember, it's your space and

In [20]:
print(seo_vars)

SEO Keyphrase: "Color Schemes and Patterns in Interior Design"

Meta Description: "Discover how to master the use of color schemes and patterns in your home's interior design. Learn about the theory, application, and impact of colors and patterns in creating spaces that reflect your personal style and mood."


In [21]:
print(input)

How to play with different color schemes and patterns to give rooms personality.
